EXTRACCIÓN DE DATOS PARA ENTRENAMIENTO DE REDES NEURONALES

Agregado del número total de delitos por cada colonia de la Ciudad de México desde 2016 hasta abril del 2025

Se dispone de la base de datos oficial de la Fiscalia General de Juzticia de la Ciudad de México con datos del Secretariado Ejecutivo de Sistema Nacional de Seguridad Pública

In [1]:
import pandas as pd
import numpy as np

def procesar_datos_homicidios(archivo_entrada, archivo_salida='/home/adonnay_bazaldua/Documentos/GitHub/Proyectos-Don-Mike/databases/homicidios_final.csv'):

    try:
        df = pd.read_csv(archivo_entrada)
        
        # Mostrar información básica del dataset
        print(f"Dimensiones del dataset: {df.shape}")
        print(f"Columnas disponibles: {df.columns.tolist()}")
        
        # Verificar que las columnas necesarias existan
        columnas_requeridas = ['anio_inicio', 'alcaldia_hecho', 'delito', 'colonia_hecho']
        columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
        
        if columnas_faltantes:
            print(f"Advertencia: Las siguientes columnas no se encontraron: {columnas_faltantes}")
            print("Columnas disponibles:", df.columns.tolist())
            return
        
        # Mostrar cantidad única de colonias
        print(f"\nCantidad de colonias únicas: {df['colonia_hecho'].nunique()}")
        print("Primeras 10 colonias:")
        print(df['colonia_hecho'].unique()[:10])
        
        # Limpiar datos (eliminar valores nulos en columnas clave)
        df_limpio = df.dropna(subset=['anio_inicio', 'alcaldia_hecho', 'delito'])
        print(f"\nRegistros después de limpiar datos faltantes: {len(df_limpio)}")
        
        # Agrupar por año y alcaldía, contar delitos
        print("\nAgrupando datos por año y alcaldía...")
        df_agrupado = (df_limpio.groupby(['anio_inicio', 'alcaldia_hecho'])
                      .agg({
                          'delito': 'count'  # Contar número de delitos
                      })
                      .reset_index())
        
        # Renombrar la columna de conteo
        df_agrupado.rename(columns={'delito': 'total_delitos'}, inplace=True)
        
        # Mostrar información del dataset agrupado
        print(f"Registros en el dataset agrupado: {len(df_agrupado)}")
        print(f"Años únicos: {sorted(df_agrupado['anio_inicio'].unique())}")
        print(f"Alcaldías únicas: {df_agrupado['alcaldia_hecho'].nunique()}")
        
        # Mostrar estadísticas básicas
        print(f"\nEstadísticas de delitos por grupo:")
        print(f"Total de delitos: {df_agrupado['total_delitos'].sum()}")
        print(f"Promedio de delitos por grupo: {df_agrupado['total_delitos'].mean():.2f}")
        print(f"Máximo de delitos en un grupo: {df_agrupado['total_delitos'].max()}")
        print(f"Mínimo de delitos en un grupo: {df_agrupado['total_delitos'].min()}")
        
        # Mostrar los 10 registros con más delitos
        print(f"\nTop 10 combinaciones año-alcaldía con más delitos:")
        top_10 = df_agrupado.nlargest(10, 'total_delitos')
        print(top_10.to_string(index=False))
        
        # Guardar el archivo procesado
        df_agrupado.to_csv(archivo_salida, index=False, encoding='utf-8')
        print(f"\nArchivo guardado exitosamente como: {archivo_salida}")
        
        # Mostrar preview del archivo guardado
        print(f"\nPreview del archivo guardado (primeras 10 filas):")
        print(df_agrupado.head(10).to_string(index=False))
        
        return df_agrupado
        
    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo {archivo_entrada}")
    except Exception as e:
        print(f"Error al procesar el archivo: {str(e)}")
        return None

def analisis_adicional(df_agrupado):
    """
    Realiza análisis adicional de los datos agrupados
    """
    if df_agrupado is None:
        return
    
    print("\n" + "="*50)
    print("ANÁLISIS ADICIONAL")
    print("="*50)
    
    # Análisis por año
    delitos_por_anio = df_agrupado.groupby('anio_inicio')['total_delitos'].sum().sort_index()
    print("\nTotal de delitos por año:")
    for anio, total in delitos_por_anio.items():
        print(f"{anio}: {total} delitos")
    
    # Análisis por alcaldía
    delitos_por_alcaldia = df_agrupado.groupby('alcaldia_hecho')['total_delitos'].sum().sort_values(ascending=False)
    print(f"\nTop 5 alcaldías con más delitos:")
    for alcaldia, total in delitos_por_alcaldia.head().items():
        print(f"{alcaldia}: {total} delitos")

# Función principal
def main():
    # Especifica aquí el nombre de tu archivo CSV
    archivo_entrada = "/home/adonnay_bazaldua/Documentos/GitHub/Proyectos-Don-Mike/databases/homicidios_final.csv"  # Cambia este nombre por el de tu archivo
    archivo_salida = "homicidios_agrupados.csv"
    
    print("PROCESADOR DE DATOS DE HOMICIDIOS")
    print("="*40)
    
    # Procesar los datos
    df_resultado = procesar_datos_homicidios(archivo_entrada, archivo_salida)
    
    # Realizar análisis adicional
    if df_resultado is not None:
        analisis_adicional(df_resultado)

# Ejecutar el script
if __name__ == "__main__":
    main()

PROCESADOR DE DATOS DE HOMICIDIOS
Dimensiones del dataset: (13854, 22)
Columnas disponibles: ['Unnamed: 0', 'anio_inicio', 'mes_inicio', 'fecha_inicio', 'hora_inicio', 'anio_hecho', 'mes_hecho', 'fecha_hecho', 'hora_hecho', 'delito', 'categoria_delito', 'competencia', 'fiscalia', 'agencia', 'unidad_investigacion', 'colonia_hecho', 'colonia_catalogo', 'alcaldia_hecho', 'alcaldia_catalogo', 'municipio_hecho', 'latitud', 'longitud']

Cantidad de colonias únicas: 1304
Primeras 10 colonias:
['NATIVITAS' 'SAN PEDRO ZACATENCO' 'PARAJE TETENCO'
 'SAN MATEO TLALTENANGO' 'ZEDEC SANTA FE' 'PUEBLO DE SAN PABLO TEPETLAPA'
 'SAN FRANCISCO TLALTENCO' 'AMPLIACIÓN PROVIDENCIA' 'EL MOLINO'
 'AGRÍCOLA PANTITLAN']

Registros después de limpiar datos faltantes: 13853

Agrupando datos por año y alcaldía...
Registros en el dataset agrupado: 160
Años únicos: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
Alcaldías únicas: 16

Estadísticas de delitos por grupo:
Total de delitos: 13853
Promedio de

Prueba para datos por colonia 

In [2]:
import pandas as pd
import numpy as np

def procesar_datos_homicidios(archivo_entrada, archivo_salida='/home/adonnay_bazaldua/Documentos/GitHub/Proyectos-Don-Mike/databases/homicidios_final.csv'):

    try:
        df = pd.read_csv(archivo_entrada)
        
        # Mostrar información básica del dataset
        print(f"Dimensiones del dataset: {df.shape}")
        print(f"Columnas disponibles: {df.columns.tolist()}")
        
        # Verificar que las columnas necesarias existan
        columnas_requeridas = ['anio_inicio', 'alcaldia_hecho', 'delito', 'colonia_hecho']
        columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
        
        if columnas_faltantes:
            print(f"Advertencia: Las siguientes columnas no se encontraron: {columnas_faltantes}")
            print("Columnas disponibles:", df.columns.tolist())
            return
        
        # Mostrar cantidad única de colonias
        print(f"\nCantidad de colonias únicas: {df['colonia_hecho'].nunique()}")
        print("Primeras 10 colonias:")
        print(df['colonia_hecho'].unique()[:10])
        
        # Limpiar datos (eliminar valores nulos en columnas clave)
        df_limpio = df.dropna(subset=['anio_inicio', 'colonia_hecho', 'delito'])
        print(f"\nRegistros después de limpiar datos faltantes: {len(df_limpio)}")
        
        # Agrupar por año y alcaldía, contar delitos
        print("\nAgrupando datos por año y alcaldía...")
        df_agrupado = (df_limpio.groupby(['anio_inicio', 'colonia_hecho'])
                      .agg({
                          'delito': 'count'  # Contar número de delitos
                      })
                      .reset_index())
        
        # Renombrar la columna de conteo
        df_agrupado.rename(columns={'delito': 'total_delitos'}, inplace=True)
        
        # Mostrar información del dataset agrupado
        print(f"Registros en el dataset agrupado: {len(df_agrupado)}")
        print(f"Años únicos: {sorted(df_agrupado['anio_inicio'].unique())}")
        print(f"Alcaldías únicas: {df_agrupado['colonia_hecho'].nunique()}")
        
        # Mostrar estadísticas básicas
        print(f"\nEstadísticas de delitos por grupo:")
        print(f"Total de delitos: {df_agrupado['total_delitos'].sum()}")
        print(f"Promedio de delitos por grupo: {df_agrupado['total_delitos'].mean():.2f}")
        print(f"Máximo de delitos en un grupo: {df_agrupado['total_delitos'].max()}")
        print(f"Mínimo de delitos en un grupo: {df_agrupado['total_delitos'].min()}")
        
        # Mostrar los 10 registros con más delitos
        print(f"\nTop 10 combinaciones año-alcaldía con más delitos:")
        top_10 = df_agrupado.nlargest(10, 'total_delitos')
        print(top_10.to_string(index=False))
        
        # Guardar el archivo procesado
        df_agrupado.to_csv(archivo_salida, index=False, encoding='utf-8')
        print(f"\nArchivo guardado exitosamente como: {archivo_salida}")
        
        # Mostrar preview del archivo guardado
        print(f"\nPreview del archivo guardado (primeras 10 filas):")
        print(df_agrupado.head(10).to_string(index=False))
        
        return df_agrupado
        
    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo {archivo_entrada}")
    except Exception as e:
        print(f"Error al procesar el archivo: {str(e)}")
        return None

def analisis_adicional(df_agrupado):
    """
    Realiza análisis adicional de los datos agrupados
    """
    if df_agrupado is None:
        return
    
    print("\n" + "="*50)
    print("ANÁLISIS ADICIONAL")
    print("="*50)
    
    # Análisis por año
    delitos_por_anio = df_agrupado.groupby('anio_inicio')['total_delitos'].sum().sort_index()
    print("\nTotal de delitos por año:")
    for anio, total in delitos_por_anio.items():
        print(f"{anio}: {total} delitos")
    
    # Análisis por alcaldía
    delitos_por_alcaldia = df_agrupado.groupby('colonia_hecho')['total_delitos'].sum().sort_values(ascending=False)
    print(f"\nTop 5 alcaldías con más delitos:")
    for alcaldia, total in delitos_por_alcaldia.head().items():
        print(f"{alcaldia}: {total} delitos")

# Función principal
def main():
    # Especifica aquí el nombre de tu archivo CSV
    archivo_entrada = "/home/adonnay_bazaldua/Documentos/GitHub/Proyectos-Don-Mike/databases/homicidios_final.csv"  # Cambia este nombre por el de tu archivo
    archivo_salida = "homicidios_agrupados.csv"
    
    print("PROCESADOR DE DATOS DE HOMICIDIOS")
    print("="*40)
    
    # Procesar los datos
    df_resultado = procesar_datos_homicidios(archivo_entrada, archivo_salida)
    
    # Realizar análisis adicional
    if df_resultado is not None:
        analisis_adicional(df_resultado)

# Ejecutar el script
if __name__ == "__main__":
    main()

PROCESADOR DE DATOS DE HOMICIDIOS
Dimensiones del dataset: (13854, 22)
Columnas disponibles: ['Unnamed: 0', 'anio_inicio', 'mes_inicio', 'fecha_inicio', 'hora_inicio', 'anio_hecho', 'mes_hecho', 'fecha_hecho', 'hora_hecho', 'delito', 'categoria_delito', 'competencia', 'fiscalia', 'agencia', 'unidad_investigacion', 'colonia_hecho', 'colonia_catalogo', 'alcaldia_hecho', 'alcaldia_catalogo', 'municipio_hecho', 'latitud', 'longitud']

Cantidad de colonias únicas: 1304
Primeras 10 colonias:
['NATIVITAS' 'SAN PEDRO ZACATENCO' 'PARAJE TETENCO'
 'SAN MATEO TLALTENANGO' 'ZEDEC SANTA FE' 'PUEBLO DE SAN PABLO TEPETLAPA'
 'SAN FRANCISCO TLALTENCO' 'AMPLIACIÓN PROVIDENCIA' 'EL MOLINO'
 'AGRÍCOLA PANTITLAN']

Registros después de limpiar datos faltantes: 13845

Agrupando datos por año y alcaldía...
Registros en el dataset agrupado: 5835
Años únicos: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
Alcaldías únicas: 1304

Estadísticas de delitos por grupo:
Total de delitos: 13845
Promedio

comprobacion de resultados

In [17]:
df = pd.read_csv('/home/adonnay_bazaldua/Documentos/GitHub/Proyectos-Don-Mike/scrips/homicidios_agrupados.csv')
print(df.head())

print(df['colonia_hecho' == 2023].unique())

   anio_inicio    colonia_hecho  total_delitos
0         2016  12 DE DICIEMBRE              1
1         2016     15 DE AGOSTO              1
2         2016     2 DE OCTUBRE              1
3         2016  20 DE NOVIEMBRE              4
4         2016      25 DE JULIO              3


KeyError: False